In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

In [27]:
# Replace these with your ResearchGate login credentials and search query
USERNAME = "kaiyisun@usc.edu"
PASSWORD = "981130aaBC871130/"

In [28]:
def encodeString(string):
    newStr = string.encode(encoding = 'UTF-8', errors = 'strict')
    return newStr

def makefileName(title):
    return title.replace("/"," ").replace(" ","_")

In [29]:
class Paper:
    def __init__(self, doi, codedDOI, title, authors):
        self.doi = doi
        self.encodedDOI = codedDOI
        self.title = title
        self.authors = authors
        self.table = None

    def getCoded(self):
        return self.encodedDOI

    def getContent(self):
        return self.table

    def getAuthors(self):
        return self.authors

    def getTitle(self):
        return self.title

    def getDOI(self):
        return self.doi
    
    def setContent(self, table):
        self.table = table
            

papers = {}
df = pd.read_csv("updated_bik_rg.csv")
rows = df.shape[0]
for i in range(rows):
    test_doi = df.loc[i, ["DOI"]][0]
    title = df.loc[i, ["Title"]][0]
    authors = df.loc[i, ['Authors']][0]
    papers[test_doi] = Paper(test_doi, encodeString(test_doi),title, authors)

In [30]:
def login(driver, username, password):
    # Fill in the login form and submit
    email_input = driver.find_element(By.NAME,"login")
    password_input = driver.find_element(By.NAME,"password")
    email_input.send_keys(username)
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    return driver

def searchByDOI(driver, doi):
    search_input = driver.find_element(By.ID, "header-search-action")
    search_input.send_keys(doi)
    search_input.send_keys(Keys.RETURN)
    return driver

def getResearchTable(driver, title):
    try:
        TABLE_HEADERS = ['Research Interest Score','Citations','Recommendations','Reads']
        research_table = driver.find_element(By.CLASS_NAME, "research-detail").text.split('\n')
        store_next_line = False
        prev_header = ""
        table_data = {}
        # Read Table
        for line in research_table:
            if (store_next_line):
                table_data[prev_header] = line
                store_next_line = False
                prev_header = ""
            elif (line in TABLE_HEADERS):
                table_data[line] = ""
                store_next_line = True
                prev_header = line
        return table_data
    except Exception as e:
        print(f"An error occurred: {e}")
        return False    

def retrievePaper(driver, doi, title):
    try: 
        driver = searchByDOI(driver, doi)
        time.sleep(3)
        table = getResearchTable(driver, title)
        if table != False:
            return table
        else:
            print(f"Failed to get table {doi}")
            return None
    except Exception as e:
        print(f"An error for {doi} occurred: {e}")

In [31]:
try:
    # Create a new instance of the browser driver (e.g., Chrome)
    driver = uc.Chrome()
    # Open ResearchGate login page
    driver.get("https://www.researchgate.net/login")
    driver = login(driver, USERNAME, PASSWORD)
    time.sleep(1)

    for doi, obj in papers.items():
        table = retrievePaper(driver, obj.getDOI(), obj.getTitle())
        if table:
            obj.setContent(table)
        else:
            print(f"Couldn't set table for {doi}")
except Exception as err:
    print(f"Failed for {doi}\nReason: {err}")
finally:
    # Close the browser
    driver.quit()

An error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":".research-detail"}
  (Session info: chrome=115.0.5790.102); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00BEA813+48355]
	(No symbol) [0x00B7C4B1]
	(No symbol) [0x00A85358]
	(No symbol) [0x00AB09A5]
	(No symbol) [0x00AB0B3B]
	(No symbol) [0x00ADE232]
	(No symbol) [0x00ACA784]
	(No symbol) [0x00ADC922]
	(No symbol) [0x00ACA536]
	(No symbol) [0x00AA82DC]
	(No symbol) [0x00AA93DD]
	GetHandleVerifier [0x00E4AABD+2539405]
	GetHandleVerifier [0x00E8A78F+2800735]
	GetHandleVerifier [0x00E8456C+2775612]
	GetHandleVerifier [0x00C751E0+616112]
	(No symbol) [0x00B85F8C]
	(No symbol) [0x00B82328]
	(No symbol) [0x00B8240B]
	(No symbol) [0x00B74FF7]
	BaseThreadInitThunk [0x76C07D59+25]
	RtlInitializeExceptionChain [0x76F8B79B+107]
	RtlClearBits [0x76F

In [32]:
df_stats_paper = []
for doi, obj in papers.items():
    if obj.getContent() == None:
        obj.setContent({'DOI':obj.getDOI(),'Authors':obj.getAuthors(),'Title': obj.getTitle(),'Research Interest Score': 'missing', 'Citations': 'missing', 'Recommendations': 'missing', 'Reads': 'missing'})
        df_stats_paper.append(pd.DataFrame(obj.getContent(), index=[0]))
    else:
        obj.getContent().update({'DOI':obj.getDOI(),'Authors':obj.getAuthors(),'Title': obj.getTitle()})
        df_stats_paper.append(pd.DataFrame(obj.getContent(), index=[0]))

In [39]:
paper_information = pd.concat(df_stats_paper, ignore_index=True)

In [42]:
paper_information.to_csv("information.csv")